In [8]:
# mhcflurry_cloud example
# Runs cross validation over a few models.
#
import mhcflurry_cloud
import mhcflurry
import fancyimpute
import distributed
import distributed.joblib
import joblib

In [3]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=2, scheduler_port=0)

In [10]:
joblib.parallel.DEFAULT_BACKEND = 'distributed'

'multiprocessing'

In [5]:
with joblib.parallel_backend('distributed', scheduler_host='localhost:8786')

In [ ]:
# Load some data.
train_data = mhcflurry.dataset.Dataset.from_csv("../test/data/bdata.2009.mhci.public.1.txt")
train_data

In [ ]:
# Generate cross validation folds, with imputation.
imputer = fancyimpute.MICE(n_imputations=2, n_burn_in=1, n_nearest_columns=25)

folds = mhcflurry_cloud.cross_validation_folds(
    train_data,
    n_folds=3,
    imputer=imputer,
    drop_similar_peptides=True,
    alleles=["HLA-A0201", "HLA-A0202"],
    n_jobs=-1,
    verbose=5,
)
folds

In [ ]:
# Make some models.
models = mhcflurry_cloud.models_grid(
    activation=["tanh", "relu"],
    layer_sizes=[[4]],
    embedding_output_dim=[8],
    n_training_epochs=[3])
models

In [ ]:
# Train and test models on the folds.
df = mhcflurry_cloud.train_across_models_and_folds(folds, models)
df